# Embedding
![rag_embedding](figures/rag_embedding.png)

- 분할된 텍스트를 벡터 표현(임베딩 벡터)으로 변환한다.
- LangChain은 OpenAI, HuggingFace 등 다양한 임베딩 모델을 지원하며, 동일한 인터페이스로 사용할 수 있다.
- [임베딩모델의 메서드](https://api.python.langchain.com/en/latest/embeddings/langchain_core.embeddings.embeddings.Embeddings.html#langchain_core.embeddings.embeddings.Embeddings)

    - **`embed_documents(texts: List[str])`**
        - 여러 문서를 받아 벡터화(임베딩)한다.
        - Context를 벡터화 할 때 사용한다.
    - **`embed_query(text: str)`**
        - 하나의 문자열(문서)을 받아 벡터화한다.
        - Query를 벡터화 할 때 사용한다.


In [1]:
docs = [
        "이 강아지 품종은 진도개 입니다. 국제 표준으로 중대형견으로 분류되며 다리가 길어 체고가 높은 편에 속합니다.",
        "日本の市内バスの運賃は主に距離によって決まり、地域やバス会社によって異なる場合があります",                  # 일본의 시내버스 요금은 주로 거리에 따라 결정되며, 지역 및 버스 회사에 따라 다를 수 있습니다.
        "Bus fares in the United States vary from city to city, but are generally around $2.90 for a regular bus.",  # 미국의 버스 요금은 도시마다 다르지만, 일반적으로 정기 버스의 경우 2.90달러 정도입니다.
        "광역버스 요금은 일반 3000원, 청소는 1800원, 어린이 1500원 입니다.", 
]

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# OpenAI의 Embedding 모델
from langchain_openai import OpenAIEmbeddings

e_model_id = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=e_model_id)

In [4]:
# Ollama Embedding Model
from langchain_ollama import OllamaEmbeddings

e_model_id = "bge-m3"
embedding_model = OllamaEmbeddings(model=e_model_id)

In [5]:
# 문서들을 Embedding
embedded_docs = embedding_model.embed_documents(docs)
embedded_docs

[[-0.041745096,
  -0.037320063,
  -0.0506333,
  0.020692611,
  -0.004415001,
  -0.019825177,
  0.01023839,
  0.018626416,
  -0.010843215,
  -0.023798939,
  -0.018735943,
  0.033569198,
  -0.023516558,
  0.009662245,
  0.014805366,
  0.002584035,
  0.010072284,
  -0.03564057,
  0.038308267,
  0.009824096,
  -0.026892466,
  0.02649794,
  -0.016067203,
  -0.04814806,
  -0.02642698,
  -0.0023369615,
  0.041257802,
  0.0047604768,
  0.07869674,
  -0.013357145,
  0.012173146,
  0.042103127,
  0.022395685,
  -0.03585081,
  0.038327616,
  -0.049167592,
  -0.011206768,
  -0.0070267185,
  -0.029203342,
  0.004924797,
  -0.011683542,
  -0.057165135,
  0.032162413,
  -0.004619371,
  0.0069473325,
  -0.003149003,
  -0.014400732,
  -0.030371744,
  -0.030845873,
  -0.007607784,
  0.019497214,
  -0.002247219,
  0.020616662,
  0.02498467,
  0.041888762,
  0.027442241,
  0.019704344,
  -0.026938235,
  -0.011060066,
  0.052702785,
  -0.029018557,
  0.061822657,
  0.0126074115,
  -0.03254941,
  -0.0095195

In [6]:
type(embedded_docs), type(embedded_docs[0])

(list, list)

list의 shape을 보고 싶은데 볼 수가 없거등요.. 그러니 numpy를 이용합니다.

In [7]:
import numpy as np
np.shape(embedded_docs)
# (4: 문서 개수, 1536: 개별 문서의 임베딩 벡터의 차원 수)

(4, 1024)

개별 문서의 임베딩 벡터의 차원 수를 기억해둬야 함. 벡터 DB에게 알려줘야 함!

In [8]:
import numpy as np

def cosine_similarity(v1:np.ndarray|list, v2:np.ndarray|list)->float:
    # v1과 v2의 코사인 유사도를 계산
    v1 = np.array(v1)
    v2 = np.array(v2)
    return (v1 @ v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [9]:
# 질문 -> Embedding Vector로 변환
#      -> 문서들의 임베딩 벡터들과 유사도를 계산
#      -> 유사도가 높은 순으로 반환

query = "당신이 좋아하는 동물은 무엇인가요?"
embedded_query = embedding_model.embed_query(query)
np.shape(embedded_query), type(embedded_query)

((1024,), list)

1차원의 (1536,)이 나온다.    
하나의 문장만 넣어서 1차원이 나옴.

In [10]:
# embedded_query와 embedding_docs 간의 유사도 계산
for i, ev in enumerate(embedded_docs):
    print(f"{i+1}. {cosine_similarity(ev, embedded_query)}")

1. 0.44087571038719436
2. 0.2017614667364376
3. 0.22486248915477347
4. 0.26833623899922326


In [ ]:
%pip install langchain-ollama

In [ ]:
%pip install langchain-huggingface

In [28]:
# HuggingFace Embedding Model
# Hugging-hub : Model > task - NLP > Feature Extraction
from langchain_huggingface import HuggingFaceEmbeddings

e_model_id = "intfloat/multilingual-e5-large-instruct"
embedding_model = HuggingFaceEmbeddings(model=e_model_id)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Anaconda3\envs\lang_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\안수민\.cache\huggingface\hub\models--intfloat--multilingual-e5-large-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

# 벡터 데이터베이스(Vector Database)
- Embedding 된 문서를 Vector Database(Vector Store)에 저장한다.
- 이후 질문(Query)와 관련된 내용을 유사도를 이용해 검색해 질문과 함께 prompt로 만든다. (Retrieve)

![rag_vector_store](figures/rag_vector_store.png)

## 벡터 데이터베이스란
- 벡터 임베딩을 저장하고 관리하는 데이터베이스를 의미한다.
- 모든 데이터는 적절한 임베딩 모델을 활용하면 임베딩 벡터로 변환할 수 있다. 이렇게 변환된 임베딩 벡터를 벡터 데이터베이스에 저장하면 **임베딩 벡터 간의 거리 계산을 통해 데이터 간 유사도를 검색할 수 있다.**
    - **이미지, 텍스트, 음성 등 비정형 데이터**를 임베딩 모델로 **벡터화한 뒤 데이터베이스에 저장**한다.
    - 벡터 간의 **유사도 계산**을 통해 연관성 있는 데이터나 유사한 데이터를 효과적으로 검색할 수 있다.
    - 좋은 검색 결과를 위해서는 벡터의 품질이 중요하다. 그래서 **임베딩 모델(Embedding Model)을 잘 선택하는 것이 중요**하다.
- 벡터 데이터베이스는 이러한 벡터 간 거리 계산에 특화된 데이터베이스다.

## 주요 특징

- **고차원 벡터 저장**
  -  벡터 데이터베이스는 수백에서 수천 차원에 이르는 벡터 데이터를 효율적으로 저장하고 관리한다. 
  -  전통적인 데이터베이스로는 어려운 고차원 벡터 간 유사도 검색을 효율적으로 수행한다.
- **유사성 기반 검색**
  -  벡터 간의 거리를 측정하여 유사한 데이터를 빠르게 검색할 수 있다. 
  -  일반적으로 사용되는 거리계산기법은 다음과 같다.
     - 코사인 유사도(Cosine Similarity)
     - 유클리드 거리(Euclidean Distance)
     - 맨하탄 거리(Manhattan Distance) 
- 비정형 데이터 처리: 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 변환하여 저장하고, 이러한 데이터를 효과적으로 검색할 수 있다.

## 벡터 데이터베이스와 딥러닝
- 벡터 데이터베이스는 딥러닝 기술의 발전과 깊은 관련이 있다.
- 딥러닝 모델은 학습 과정에서 데이터의 특징을 추출하는 방법을 함께 학습한다. 충분한 데이터를 학습한 딥러닝 모델은 **데이터의 특성을 설명하는 특성 벡터(feature vector)를 효과적으로 생성**할 수 있다.
- 이때 추출된 특성 벡터는 고차원 데이터(RAW Data)를 저차원 공간에서 표현한 **임베딩 벡터**다.
    - > **임베딩**은 고차원 데이터를 저차원 공간으로 변환하여 표현하는 방법으로, 정보 손실을 최소화하면서 데이터 간의 의미 있는 관계를 벡터 공간에서 유지한다.
- 딥러닝 모델로 추출한 데이터의 특징(feature vector)을 임베딩 공간에 배치하면, 비슷한 데이터는 가까이, 그렇지 않은 데이터는 멀리 배치된다.
- 이러한 특성을 활용하면 임베딩 벡터 간의 거리를 계산해 유사한 데이터를 효과적으로 검색할 수 있다. 벡터 데이터베이스는 이러한 임베딩 벡터의 특성을 기반으로 개발되었다.
- 딥러닝 기술의 발전과 폭넓은 활용으로 임베딩 데이터의 사용이 증가하면서, 이를 저장하고 관리하는 기능에 특화된 데이터베이스에 대한 수요도 증가해 다양한 벡터 데이터베이스가 등장했다.

## 벡터 데이터베이스의 주요 기능
1. **저장**  
   - 이미지, 텍스트, 음성 등 **비정형 데이터**를 임베딩 모델을 통해 벡터로 변환한 뒤 벡터 데이터베이스에 저장한다.
2. **검색**  
   - 검색하려는 데이터를 임베딩 모델로 변환한 뒤, 벡터 데이터베이스에서 **유사도를 기반**으로 검색한다.
3. **결과 반환**  
   - 벡터 데이터베이스는 저장된 벡터 중 검색 쿼리 임베딩과 가장 가까운 벡터를 찾아 반환한다.

## LLM과 벡터 데이터베이스
- ChatGPT(LLM)의 등장 이후 벡터 데이터베이스는 폭발적인 주목을 받았다.
- 임베딩 벡터의 유사도를 기반으로 문서를 검색하는 RAG(Relevant Augmented Generation) 기술은 LLM의 환각(할루시네이션) 현상을 줄이고, LLM을 추가 학습하지 않고도 최신 정보를 효율적으로 활용할 수 있는 핵심 기법으로 자리 잡았다.
   


## 벡터 데이터베이스 종류
![img](figures/vector_database.png)

<<https://blog.det.life/why-you-shouldnt-invest-in-vector-databases-c0cd3f59d23c>>

### 주요 벡터 데이터베이스 종류
- **Pinecone**
    - 클라우드 기반의 완전 관리형 벡터 데이터베이스 서비스로, 간단한 API를 통해 벡터 데이터를 관리할 수 있다.  
    - 자동 확장성과 고가용성을 제공하며, 실시간 데이터 수집과 유사성 검색에 최적화되어 있다.
    - 가장 쉽게 시작할 수 있는 관리형 서비스를 제공한다.
- **Chroma**
    - 벡터 임베딩을 효율적으로 저장하고 검색할 수 있는 오픈소스 데이터베이스로, AI 및 머신러닝 애플리케이션에 최적화되어 있다.
    - 대규모 임베딩 저장에 최적화되어 있다.
- **FAISS**
    - Facebook AI에서 개발한 고성능 벡터 검색 라이브러리로, 고차원 벡터의 효율적인 유사성 검색을 위해 최적화되어 있다.
    - GPU를 활용해 계산 성능을 높이며, 벡터 양자화 기술을 활용하여 메모리 사용을 최적화한다.
    - 근사 최근접 이웃 검색(ANNS)에 최적화되어 있다.
- **Milvus**
    - 오픈소스 벡터 데이터베이스로, 대규모 벡터 데이터를 효율적으로 저장하고 검색할 수 있다.  
    - 분산 아키텍처를 채택하여 확장성이 뛰어나며, IVF_PQ, DiskANN 등 다양한 인덱싱 알고리즘을 지원한다.
    - 대규모 데이터셋 처리에 가장 적합한 솔루션이다.
- **Weaviate**
    - 오픈소스 벡터 데이터베이스로, 텍스트, 이미지, 오디오 등 다양한 비정형 데이터를 벡터로 저장하고 검색할 수 있다.  
    - GraphQL API를 통해 접근 가능하며, 내장된 머신러닝 모듈을 통해 가장 강력한 의미론적 검색 기능을 제공한다.
- **Qdrant**
    - Rust로 개발된 고성능 벡터 검색 엔진으로, 실시간 근사 최근접 이웃 검색을 제공한다.  
    - 추천 시스템에 특화되어 있으며, 벡터 임베딩 저장과 유사도 쿼리를 효율적으로 수행한다.
- **Elasticsearch**
    - HNSW 알고리즘을 사용하여 벡터 검색을 구현하는 검색 엔진이다.
    - 전통적인 검색 기능과 벡터 검색을 효과적으로 결합할 수 있어, 하이브리드 검색에 가장 적합하다.
- **PGVector**
    - PostgreSQL의 확장 모듈로, 벡터 데이터를 저장하고 유사성 검색을 수행할 수 있게 해준다.  
    - SQL과 통합된 벡터 연산이 가능하며, L2 거리, 코사인 거리, 내적 등 다양한 거리 측정 방식을 지원한다.


# Langchain - Vector Store 연동 
- Langchain은 다양한 벡터 데이터베이스와 연동할 수 있다.
- 벡터 데이터베이스 마다 API가 다르기 때문에, Langchain을 사용하면 동일한 interface로 사용할 수 있다.

## **VectorStore**
- Langchain이 지원하는 모든 벡터 데이터베이스는 **VectorStore** 인터페이스를 구현한다.
- 그래서 Langchain에서는 벡터 데이터베이스를 **Vector Store** 라고 한다.
- https://python.langchain.com/docs/integrations/vectorstores/

### Vector Store 연결
- Vector DB와 연결하는 메소드
- `VectorStore.from_document()`
  - Document들을 insert 하면서 연결.
  - Database가 있으면 연결, 없으면 생성하면서 연결한다.
  - Parameter
    - documents: insert할 문서들을 list[Document]로 전달.
    - embedding model
    - vector db에 연결하기 위한 설정들을 넣어준다.
-`VectorStore()`
  - vector db와 연결만 한다.
  - Database가 있으면 연결, 없으면 생성하면서 연결한다.
  - Parameter
    - embedding model
    - vector db에 연결하기 위한 설정들을 넣어준다.
## InMemoryVectorStore
- langchain에서 제공하는 메모리 기반 벡터 데이터베이스이다.
- Data들을 Dictionary를 사용해 메모리에 저장하며, 검색 할 때 코사인 유사도(cosine similarity)를 계산하여 조회한다.

In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# vectorStore 생성시 Embedding 모델을 넣어 생성한다.
vector_store = InMemoryVectorStore(
    embedding=embedding_model
)

임베딩 모델 이거 쓸거야. `OpenAIEmbeddings()`

vector DB(=vector store)를 정의할 거야. 임베딩 모델은 위에서 정의한 거야. `InMemoryVectorStore()`

In [13]:
# 문서 정의
from langchain_core.documents import Document
d1 = Document(id="1", # 문서 ID (식별자) 
              page_content="Apple, Pear, WaterMelon", # 문서 내용
              metadata={"category": "fruit"} # 문서 정보
              )
d2 = Document(id="2", # 문서 ID (식별자) 
              page_content="Python, C++, Java, C#", # 문서 내용
              metadata={"category": "IT"} # 문서 정보
              )
d3 = Document(id="3", # 문서 ID (식별자) 
              page_content="Football, Baseball, Basketball", # 문서 내용
              metadata={"category": "Sports"} # 문서 정보
              )

docs = [d1, d2, d3]
# Vector DB에 저장
vector_store.add_documents(documents=docs)

['1', '2', '3']

문서를 정의하고, 벡터 DB에 문서를 추가할 거야. `vector_store.add_documents()`

참고: content만 임베딩 시킨다.

In [14]:
# DB와 연결하면서 document들을 insert/upsert

vector_store2 = InMemoryVectorStore.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [15]:
# 검색 = Query와 유사한 문서를 Vector Store에서 찾기
query = "SQL"
query = "야구"

# result = vector_store.similarity_search(
result = vector_store.similarity_search_with_score( # 검색한 결과 + 우사도 점수까지 반환
    query=query,    # 찾을 질문
    k=2             # 몇 개 문서 찾을지 지정
)

Query와 유사한 문서를 Vector Store에서 찾을 거야.   
이때 함수는 `vector_store.similarity_search()`나 `vector_store.similarity_search_with_score()`

In [16]:
result # query = "SQL"

[(Document(id='3', metadata={'category': 'Sports'}, page_content='Football, Baseball, Basketball'),
  0.5146885017042375),
 (Document(id='2', metadata={'category': 'IT'}, page_content='Python, C++, Java, C#'),
  0.16704293238470305)]

SQL이 프로그래밍 언어인 걸 알기 때문에 위와 같은 결과가 나옴


In [17]:
result # query = "야구"

[(Document(id='3', metadata={'category': 'Sports'}, page_content='Football, Baseball, Basketball'),
  0.5146885017042375),
 (Document(id='2', metadata={'category': 'IT'}, page_content='Python, C++, Java, C#'),
  0.16704293238470305)]

In [18]:
result # query = "SQL", with score

[(Document(id='3', metadata={'category': 'Sports'}, page_content='Football, Baseball, Basketball'),
  0.5146885017042375),
 (Document(id='2', metadata={'category': 'IT'}, page_content='Python, C++, Java, C#'),
  0.16704293238470305)]

In [19]:
result # query = "야구", with score

[(Document(id='3', metadata={'category': 'Sports'}, page_content='Football, Baseball, Basketball'),
  0.5146885017042375),
 (Document(id='2', metadata={'category': 'IT'}, page_content='Python, C++, Java, C#'),
  0.16704293238470305)]

# 실습
- `data/olympic.txt`
1. text loading
2. text split
3. embedding + vector store(InMemoryVectorStore)에 저장
4. query(질의)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
# 1. 문서 Loading
# with open("o.txt") as f 해도 되나, 더 간단한 방법이 있다.
loader = TextLoader("data/olympic.txt", encoding="utf-8")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50
)
docs = loader.load_and_split(splitter)
# raw_docs = loader.load()
# docs = splitter.split_documents(raw_docs)
# load와 split을 한 번에 해주는 건 `loader.load_and_split(splitter)`
# loader. 하고 괄호 안에 splitter를 넣어준다.
print(len(docs))

61


In [25]:
# 3. Embedding + 4. Vector Store에 저장
# 두 단계는 같이 이루어짐.
vector_store = InMemoryVectorStore(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small") # 임베딩 모델
)
# 저장 (Embedding 처리 후 저장 -> vector store가 임베딩 작업을 처리한다.)
vector_store.add_documents(docs)

['46374455-ef80-4f31-8a63-85699e367032',
 '9b30531c-61f9-4485-a81e-fbb00acf4b7e',
 '747d2178-e56b-4526-9b2b-54c1ce6ef740',
 '3659e6fd-e724-4a2e-b9fe-462f6d3e9160',
 '2ea76d00-24e7-4453-89ba-49c87b5a264d',
 '5bd2bd42-a480-4dba-9d88-8a36f599ea11',
 'fa2e96db-80f0-49dc-aa10-3add63ee89f0',
 'e5f0d2ee-9302-45e9-b990-06070cfd11fe',
 '03bb07bd-0250-4238-bf49-df1d99288f3a',
 '947d72ff-6b01-44e8-ba84-fe14c0824c44',
 'ca539a1d-a2b5-4ae6-a050-0fd706249f8c',
 '570bfb9e-9e5c-4f82-8162-ed7e8b39b99b',
 '5dc787df-2764-47a8-9799-c7d979e98dfb',
 'b0533479-3fa5-4726-9662-b17c01f5a0b7',
 '27299913-a817-41f8-9639-9d5385ed85a7',
 '94b485d4-326f-4015-be64-5a5247bfea0a',
 'f8a95416-26a3-4fd6-bd6d-6564b0bcf7b8',
 '0acad70c-e56b-4e44-8787-80f5f2a658f1',
 'cafa6850-4882-426d-aa0d-8be5f790a0a7',
 '62a972df-a298-48dc-884e-0a066312ea04',
 'f8827397-39e9-40ab-ab27-245283c639e1',
 '11d8754e-9f34-4b3b-9477-97e078cbafac',
 '8ccd2607-6952-4b2c-8583-ad60c4c888dd',
 '0e58fb51-a18d-4865-a018-9f5a64c7c42a',
 '955bab60-e325-

저장하면 -> 저장한 것들의 id를 쭉 보여준다.

아래 코드와 같이 연결과 저장을 한 번에 해도 괜찮다. 

In [ ]:
# # 연결 + 저장
# v2 = InMemoryVectorStore(
#     embidding=OpenAIEmbeddings(model="text-embedding-3-small"),
#     documents=docs
# )

In [26]:
docs[0]

Document(metadata={'source': 'data/olympic.txt'}, page_content='올림픽')

검색하는 메서드는 `similarity_with_~~`였죠?

In [32]:
# 질문 -> 의미적 유사도가 높은 k개의 문서를 반환한다. 
query = input("질문:") # 올림픽 종목 알려줘
results = vector_store.similarity_search_with_score(
    query=query,
    k=5
)
results

[(Document(id='46374455-ef80-4f31-8a63-85699e367032', metadata={'source': 'data/olympic.txt'}, page_content='올림픽'),
  0.7704441665851297),
 (Document(id='fa2e96db-80f0-49dc-aa10-3add63ee89f0', metadata={'source': 'data/olympic.txt'}, page_content='고대올림픽'),
  0.5677603073483819),
 (Document(id='294416ed-e388-4174-8760-fa348347eb77', metadata={'source': 'data/olympic.txt'}, page_content='올림픽 경기 종목\n올림픽 경기 종목은 총 33개부문 52개 종목에서 약 400개의 경기로 이루어져있다. 예를 들어서 하계 올림픽 부문인 레슬링은 자유형과 그레코로만형의 두 종목으로 나뉜다. 여기에서 10경기는 남자부, 4경기는 여자부로 열리며 분류기준은 체중이다. 하계 올림픽은 26개, 동계 올림픽은 7개 부문으로 이루어져있다. 하계 올림픽에서는 육상, 수영, 펜싱, 체조가 1회 대회때부터 한번도 빠짐없이 정식종목이었으며, 동계 올림픽에서는 크로스컨트리, 피겨 스케이팅, 아이스 하키, 노르딕 복합, 스키 점프, 스피드 스케이팅이 1924년 동계 올림픽부터 빠짐없이 정식종목이었다. 배드민턴, 농구, 배구와 같은 정식종목들은 처음에는 시범종목이었으며 그 후에 정식종목으로 승인 되었다. 야구처럼 예전에는 정식종목 이었지만 지금은 정식 종목에서 빠진 종목도 있다.'),
  0.5446152941289001),
 (Document(id='94b485d4-326f-4015-be64-5a5247bfea0a', metadata={'source': 'data/olympic.txt'}, page_content='하계올림픽'),
  0.5337782171697669),
 (Document(id=

In [33]:
for result in results:
    print(result[1], result[0].page_content[:200])

0.7704441665851297 올림픽
0.5677603073483819 고대올림픽
0.5446152941289001 올림픽 경기 종목
올림픽 경기 종목은 총 33개부문 52개 종목에서 약 400개의 경기로 이루어져있다. 예를 들어서 하계 올림픽 부문인 레슬링은 자유형과 그레코로만형의 두 종목으로 나뉜다. 여기에서 10경기는 남자부, 4경기는 여자부로 열리며 분류기준은 체중이다. 하계 올림픽은 26개, 동계 올림픽은 7개 부문으로 이루어져있다. 하계 올림픽에서는 육상, 수
0.5337782171697669 하계올림픽
0.5231644370822403 올림픽은 거의 모든 국가가 참여할 정도로 규모가 커졌다. 하계 올림픽은 33개의 종목과 약 400개의 세부종목에서 13,000명이 넘는 선수들이 겨루고 그중 각 종목별 1, 2, 3위는 각각 금/은/동을 수여받는다. 전 세계 언론에서 각각 4년마다 열리는 올림픽 경기를 중계하기 때문에 이름 없는 선수가 개인적, 국가적, 세계적으로 명성을 얻을 수 있는 기회


이제, query와 추출 내용을 가지고 LLM 모델에 넣어 답변을 얻어낸다.

## MMR(최대 한계 관련성-Maximal Marginal Relevance) 알고리즘 적용
최대 한계 관련성(Maximal Marginal Relevance, MMR) 알고리즘은 정보 검색 및 요약에서 검색 결과의 **관련성**과 **다양성**을 동시에 고려하여 최적의 결과를 제공하는 방법이다. 
이 알고리즘은 사용자 쿼리와의 관련성을 최대화하면서도 중복 정보를 최소화하여 다양한 정보를 제공하는 것을 목표로 한다.

1. **관련성과 다양성의 균형 조절**: MMR은 사용자 쿼리와 문서 간의 유사성 점수와 이미 선택된 문서들과의 다양성 점수를 조합하여 각 문서의 최종 점수를 계산한다. 이를 통해 관련성이 높으면서도 중복되지 않는 문서를 선택한다.

2. **수학적 정의**
   $$
   \text{MMR} = \lambda \cdot \text{Sim}(d, Q) - (1 - \lambda) \cdot \max_{d' \in D'} \text{Sim}(d, d')
   $$

   - $\text{Sim}(d, Q)$: 문서 $d$와 쿼리 $\text{Q}$ 사이의 유사성. (문서 유사성 계산)
   - $\max_{d' \in D'} \text{Sim}(d, d')$: 문서 $d$와 이미 선택된 문서 집합 $D'$ 중 가장 유사한 문서와의 유사성. (문서 다양성 계산)
   - $\lambda$: 유사성과 다양성의 중요도를 조절하는 매개변수(parameter)
3. **적용 분야**: MMR은 정보 검색, 추천 시스템, 문서 요약 등에서 활용된다. 특히 LLM 검색에서 성능 향상이 입증되었다.

### `vectorStore.max_marginal_relevance_search()` 메소드
  - MMR 알고리즘을 적용한 검색을 수행한다.
  - **파라미터**
    - **query**: 사용자로부터 입력받은 검색 쿼리
    - **k**: 최종적으로 선택할 문서의 수
    - **fetch\_k**: MMR 알고리즘 적용 시 고려할 상위 문서의 수
    - **lambda_mult**: 쿼리와의 유사성과 선택된 문서 간의 다양성 사이의 균형을 조절하는 매개변수. $\lambda = 1$이면 유사성만 고려하고, $\lambda = 0$이면 다양성만을 최대화한다.
    - **filter**: 검색 결과를 필터링할 조건을 지정한다.


In [35]:
query = "동계 올림픽에 대해 설명해줘."
mmr_result = vector_store.max_marginal_relevance_search(
    query=query,
    k=5, # 최종 결과 문서 개수
    fetch_k=20, # 처음 검색할 문서 개수.
    lambda_mult=0.5 # 1에 가까울수록 유사성에, 0에 가까울수록 다양성을 최대화.
)
for result in mmr_result:
    print(result.page_content[:200])
    print("-"*20)

하계올림픽
--------------------
- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.
- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원
--------------------
고대 올림피아 경기가 처음 열린 시점은 보통 기원전 776년으로 인정되고 있는데, 이 연대는 그리스 올림피아에서 발견된 비문에 근거를 둔 것이다. 이 비문의 내용은 달리기 경주 승자 목록이며 기원전 776년부터 4년 이후 올림피아 경기 마다의 기록이 남겨져 있다. 고대 올림픽의 종목으로는 육상, 5종 경기(원반던지기, 창던지기, 달리기, 레슬링, 멀리뛰기)
--------------------
동계올림픽
동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기
--------------------
캐나다에서 열리는 두 번째 동계 올림픽이고, 동/하계 올림픽을 합쳐 캐나다에서 3번째로 개최되는 올림픽이다.
--------------------
